 ##### IMPORTS

In [1]:
from dmd_control import dmd
import imageio
import numpy as np
import time
from glob import glob
import os

In [2]:
XP_FOLDER = r'D:\Data\Or42b_57C10__Chronos-mVenus_rGECO'
TARGET_FOLDER = XP_FOLDER + r'\2_488_20230829_164410'
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

In [3]:
mirrors = dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


##### Calibration procedure:

- project the *calibration_mask*;
- save the image through the Labview code;
- run the *find_affine* method.

The affine transorm parameters are saved for future use.

In [4]:
mirrors.project(mirrors.calibration_mask)

Save the calibration image. Usually choosing the same folder as **mirrors.parameters['params_folder']** is a good choice to keep things tidy.
Then plug it in the following method and run it to compute the afffine transform parameters.

In [ ]:
calibration_image = \
    r'C:\Users\Light Sheet User\Documents\GitHub\DMD\camera_DMD_calibration.png'
mirrors.find_affine(calibration_image)

#### ROI selection

Use the following method to select ROIs to be stimulated by the DMD.
It is supposed to read rectangular ROIs, where you start drawing from top left corner to bottom right corner of the ROI. Different geometries are not implemented and may result in random or no pattern at all.

In [4]:
mirrors.select_ROIs()

Once the wanted ROIs are drawn, import them within the class. They are stored in order in the "rois" list and saved as .png using the save_dir attribute.

In [5]:
mirrors.import_N_rois(exp='3_test')

Found 4 ROI(s)


In case you plan to project two patterns at the same time, use the method "sum_patterns", passing a list of pattrerns to be summed as numpy matrices. It will return a new matrix that can be used for projection or for a protocol.

### PROTOCOL:
Load the images you want to use, and input how long each image should be displayed.

In [8]:
img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')
img2 = imageio.imread(mirrors.save_dir + '\\mask_2.png')
img3 = imageio.imread(mirrors.save_dir + '\\mask_3.png')
img4 = imageio.imread(mirrors.save_dir + '\\mask_4.png')

img_seq = [img1, mirrors.dark,
            img2, mirrors.dark,
             img3, mirrors.dark,
              img4, mirrors.dark,
               img3, mirrors.dark,
                img1, mirrors.dark]
img_seq_names = ['od1', 'dark',
                'od2', 'dark',
                 'od3', 'dark',
                 'od4', 'dark',
                 'od3', 'dark',
                 'od1', 'dark'] # for saving later
img_times = [5, 5,
             5, 5,
             5, 5,
             5, 5,
             5, 5,
             5, 5] # in seconds
reps = 2 # whole sequence repetition
start = time.time()
mirrors.sequence_of_single_images(img_seq, img_times, reps)
mirrors.idle()

# SAVE THE PROTOCOL
# The path to the current acquisition folder is needed in order to save the sequence of projections.
images_folder = mirrors.xp_folder + r'\2_2_20230829_171405' # <-- HERE
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + "triggers_2_numpy", 
         TM=num, images=img_seq_names, durations=img_times)

# HOW TO READ IT:
# a = np.load(images_folder + "triggers_numpy.npz")
# lst = a.files
# for item in lst:
#     print(item)
#     print(a[item])

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_12892\2800038550.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(mirrors.save_dir + '\\mask_1.png')
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_12892\2800038550.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(mirrors.save_dir + '\\mask_2.png')
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_12892\2800038550.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and mak

In [9]:
a = np.load(images_folder + "triggers_numpy.npz")
lst = a.files
for item in lst:
    print(item)
    print(a[item])

TM
558
images
['od1' 'dark' 'od2' 'dark' 'od3' 'dark' 'od4' 'dark' 'od3' 'dark' 'od1'
 'dark']
durations
[5 5 5 5 5 5 5 5 5 5 5 5]


### Close the DMD when done.

In [ ]:
mirrors.close()